In [1]:
import requests
import pandas as pd
from datetime import datetime
import os

def get_stock_data(symbol, function='TIME_SERIES_DAILY', output_size='compact'):
    """
    Fetch stock data from Alpha Vantage API
    
    Parameters:
    symbol (str): Stock symbol (e.g., 'IBM', 'AAPL')
    function (str): Alpha Vantage API function to use
    output_size (str): 'compact' returns latest 100 datapoints, 'full' returns all available data
    
    Returns:
    pandas.DataFrame: Stock data
    """
    # Load API key from environment variable
    api_key = os.getenv('ALPHA_VANTAGE_API_KEY')
    
    if not api_key:
        raise ValueError("Please set ALPHA_VANTAGE_API_KEY environment variable")
    
    base_url = 'https://www.alphavantage.co/query'
    params = {
        'function': function,
        'symbol': symbol,
        'apikey': api_key,
        'outputsize': output_size
    }
    
    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()
        
        # Extract time series data
        if function == 'TIME_SERIES_DAILY':
            time_series = data.get('Time Series (Daily)', {})
        else:
            raise ValueError(f"Function {function} not implemented in this example")
        
        # Convert to DataFrame
        df = pd.DataFrame.from_dict(time_series, orient='index')
        
        # Clean up column names and convert to numeric
        df.columns = [col.split('. ')[1] for col in df.columns]
        for col in df.columns:
            df[col] = pd.to_numeric(df[col])
            
        df.index = pd.to_datetime(df.index)
        df = df.sort_index()
        
        return df
        
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None

# Example usage
if __name__ == "__main__":

    os.environ['ALPHA_VANTAGE_API_KEY'] = 'YOUR_API_KEY' 
    

    symbol = 'AAPL'  # Example stock symbol
    df = get_stock_data(symbol)
    
    if df is not None:
        print(f"\nLatest data for {symbol}:")
        print(df.tail())
        
        # Basic statistics
        print("\nSummary statistics:")
        print(df['close'].describe())


Latest data for AAPL:
               open    high      low   close    volume
2025-02-03  229.990  231.83  225.700  228.01  73063301
2025-02-04  227.250  233.13  226.650  232.80  45067301
2025-02-05  228.530  232.67  228.270  232.47  39664989
2025-02-06  231.285  233.80  230.425  233.22  29925349
2025-02-07  232.600  234.00  227.260  227.63  39707224

Summary statistics:
count    100.000000
mean     234.085400
std        9.828695
min      216.320000
25%      227.267500
50%      231.595000
75%      240.245000
max      259.020000
Name: close, dtype: float64
